In [2]:
print("Hello")

Hello


In [4]:
!which python

/opt/conda/bin/python


In [6]:
!python --version

Python 3.11.6


In [7]:
!java -version

openjdk version "17.0.8.1" 2023-08-24
OpenJDK Runtime Environment (build 17.0.8.1+1-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 17.0.8.1+1-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [8]:
pip show pyspark

Name: pyspark
Version: 3.5.0
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /usr/local/spark/python
Requires: py4j
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Spark_home
import os
os.environ.get("SPARK_HOME")

'/usr/local/spark'

In [11]:
# java_home
os.environ.get("JAVA_HOME")

In [13]:
os.environ.get("PYTHONPATH")

'/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip:/usr/local/spark/python:'

In [14]:
import pyspark

In [17]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [18]:
spark

In [20]:
spark.stop()

In [21]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [22]:
spark

In [23]:
data = [('Ailce',1),('Job',2),('Charlie',3)]
type(data)

list

In [28]:
data1 = spark.createDataFrame(data, ['Name','Value']) # spark의 분산객체: 데이터프레임 형식
data1[0][1]

Column<'Name[1]'>

In [29]:
data1.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Ailce|    1|
|    Job|    2|
|Charlie|    3|
+-------+-----+



# RDD

In [31]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate() # 기존에 있는 거

In [35]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5])
rdd

ParallelCollectionRDD[28] at readRDDFromFile at PythonRDD.scala:289

In [36]:
data1

DataFrame[Name: string, Value: bigint]

In [38]:
rdd.take(5) # 객체를 생성 - n개를 지정

[1, 2, 3, 4, 5]

In [39]:
spark

In [41]:
# map연산
rdd_1 = rdd.map(lambda x: x*x)
rdd_1

PythonRDD[33] at RDD at PythonRDD.scala:53

In [43]:
rdd_1.take(3)

[1, 4, 9]

In [44]:
rdd.take(3)

[1, 2, 3]

In [45]:
rdd_1.collect() # 전부 다 가지고오기

[1, 4, 9, 16, 25]

In [52]:
data1.filter(data1.Name == 'Bob').show()

+----+-----+
|Name|Value|
+----+-----+
+----+-----+



In [53]:
data1.filter(data1.Value > 2).show()

+-------+-----+
|   Name|Value|
+-------+-----+
|Charlie|    3|
+-------+-----+



In [57]:
data1.createOrReplaceTempView('people') # 메모리상에 뷰 만들기

In [61]:
spark.sql('select * from people').show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Ailce|    1|
|    Job|    2|
|Charlie|    3|
+-------+-----+



In [62]:
spark.sql('select Name from people').show()

+-------+
|   Name|
+-------+
|  Ailce|
|    Job|
|Charlie|
+-------+



In [65]:
spark.sql("select * from people where name = 'Job'").show()

+----+-----+
|Name|Value|
+----+-----+
| Job|    2|
+----+-----+



# MLlib

In [67]:
from pyspark.ml.regression import LinearRegression

In [68]:
import numpy as np

In [69]:
from pyspark.ml.feature import VectorAssembler

In [71]:
data_age = [('Ailce',25),('Bob',30),('Charlie',33)]

In [73]:
data2 = spark.createDataFrame(data_age, ['Name','Age'])
data2

DataFrame[Name: string, Age: bigint]

In [75]:
assembler = VectorAssembler(inputCols = ['Age'], outputCol = 'features')

In [76]:
vector_df = assembler.transform(data2)

In [80]:
lr = LinearRegression(featuresCol = 'features',labelCol = 'Age')
model = lr.fit(vector_df)

In [82]:
pred = model.transform(vector_df)

In [83]:
pred.show()

+-------+---+--------+-----------------+
|   Name|Age|features|       prediction|
+-------+---+--------+-----------------+
|  Ailce| 25|  [25.0]|24.99999999999993|
|    Bob| 30|  [30.0]|30.00000000000001|
|Charlie| 33|  [33.0]|33.00000000000006|
+-------+---+--------+-----------------+



In [84]:
spark.stop()

# streaming

In [93]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [94]:
spark = SparkSession.builder.appName('pyspark example1').getOrCreate()

In [95]:
lines = spark.readStream.format('socket').option('host','localhost')\
.option('port',9999).load()

In [97]:
words = lines.select(explode(split(lines.value,' ')).alias('word'))

In [98]:
spark.stop()